In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import os
from pathlib import Path
import os.path as osp

C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.noijjg62emaszi6nyurl6jbkm4evbgm7.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\softwares\conda4.8.2-python3.7\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
C:\softwares\conda4.8.2-python3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\softwares\conda4.

In [2]:
#Since its a forecasting problem we will divide the data in the ratio 80:15:5 for model training,
#preparing ensemble test results and testing of ensembling results

In [3]:
data_path = osp.join(os.getcwd(),'Data')

In [4]:
dj = pd.read_csv(data_path + "/stock_indicators.csv")
news = pd.read_csv(data_path + "/news_modified.csv")

In [5]:
dj.columns

Index(['Date', 'Unnamed: 0', 'cap', 'stochaistic_k_percent', 'stochaistic_D',
       'Momentum', 'rate_of_change', 'William_R_percent', 'AD_oscillator',
       'Disparity'],
      dtype='object')

In [6]:
dj.head()

,Date,Unnamed: 0,cap,stochaistic_k_percent,stochaistic_D,Momentum,rate_of_change,William_R_percent,AD_oscillator,Disparity
0,2008-08-08,1988.0,297.580078,93.106137,13.300877,0.000000,8649.194013,0.068939,0.068939,100.000000
1,2008-08-11,1987.0,52.030273,82.307244,25.059054,48.029297,24531.588728,0.176928,0.693131,100.204235
2,2008-08-12,1986.0,-148.890625,53.109049,32.646061,-91.850585,-12675.444285,0.468910,0.000000,99.340910
3,2008-08-13,1985.0,-100.739258,30.250241,36.967524,-201.360351,-5727.522774,0.697498,-0.048157,98.800097
4,2008-08-14,1984.0,79.139649,47.569144,43.763116,-118.390624,-9811.528393,0.524309,0.693070,99.608320


In [7]:
dj['Date'] = pd.to_datetime(dj['Date']).dt.strftime('%d-%m-%Y')

In [8]:
dj = dj.drop(['Unnamed: 0'],axis=1)

In [9]:
news.head()

,Date,News
0,08-06-2008,"b'Marriage, they said, was reduced to the stat..."
1,08-06-2008,b'Nim Chimpsky: The tragedy of the chimp who t...
2,08-06-2008,"b""Canada: Beware slippery slope' to censorship..."
3,08-06-2008,b'EU Vice-President Luisa Morgantini and the I...
4,08-06-2008,"b""Israeli minister: Israel will attack Iran if..."


In [10]:
from dataPreparation import clean_text,decontracted
from utils import merge_headlines,normalize_price

In [11]:
all_headlines,dates = merge_headlines(dj,news,max_headline_length = 20,max_daily_length = 400)                    

In [12]:
df = pd.DataFrame.from_dict({'Date': dates,'News': all_headlines })

In [14]:
#here we observe that for some days in news column is empty .
#This may be due to the fact that reddit scores were low on those given dates
#However we will consider the dates only for which news are available

In [15]:
df = df.loc[df['News'] != ' ']

In [16]:
df_merge = pd.merge(df, dj, on='Date')

In [17]:
#Now we will create ensemble-train,ensemble-valid and ensemble-test data on this merged dataframe in the ratio 0.8/0.15/0.05 hence
#we will take first 90% rows for model training and evaluation next 10% for ensemble
model,ensemble = np.split(df_merge, [int(.9*len(df_merge))])

In [18]:
model

,Date,News,cap,stochaistic_k_percent,stochaistic_D,Momentum,rate_of_change,William_R_percent,AD_oscillator,Disparity
0,08-08-2008,b georgian troops retreat osettain capital pr...,297.580078,93.106137,13.300877,0.000000,8649.194013,0.068939,0.068939,100.000000
1,11-08-2008,b russia angered israeli military sale georgi...,52.030273,82.307244,25.059054,48.029297,24531.588728,0.176928,0.693131,100.204235
2,12-08-2008,b united states beats war drum iran dumps dol...,-148.890625,53.109049,32.646061,-91.850585,-12675.444285,0.468910,0.000000,99.340910
3,13-08-2008,b bush announces operation get russia grill y...,-100.739258,30.250241,36.967524,-201.360351,-5727.522774,0.697498,-0.048157,98.800097
4,14-08-2008,b poland us agree missle defense deal interes...,79.139649,47.569144,43.763116,-118.390624,-9811.528393,0.524309,0.693070,99.608320
...,...,...,...,...,...,...,...,...,...,...
2332,01-11-2018,first ever drone swarm attack struck russian ...,301.519531,98.765502,50.500099,189.310546,13406.934146,0.012345,0.973469,102.065165
2333,02-11-2018,iceland use energy mining bitcoin powering ho...,-182.128906,78.846721,56.007621,687.410156,3676.237521,0.211533,0.396275,101.223441
2334,05-11-2018,merkel rips trump pulling iran deal undermine...,191.359375,91.949116,68.584561,477.148438,5336.221853,0.080509,0.961935,101.710272
2335,06-11-2018,trump leave summit north korea early american...,335.630860,98.898451,77.663168,946.699219,2707.830455,0.011015,0.918830,101.852332


In [19]:
model.to_csv(data_path + '/model.csv')
ensemble.to_csv(data_path + '/ensemble.csv')